In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
from sklearn import preprocessing 
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train= pd.read_csv("/kaggle/input/loanprediction/train_ctrUa4K.csv")
train.profile_report()

In [ ]:
train.head()

In [ ]:
test= pd.read_csv("/kaggle/input/loanprediction/test_lAUu6dG.csv")
test.profile_report()

In [ ]:
test.head()

In [ ]:
#train['Total_income'] = train.apply(lambda row: row.ApplicantIncome(row.CoapplicantIncome), axis = 1) 


In [ ]:
train.corr()


In [ ]:
test.corr()

So, as we can see none of them has high correlation.We won't be droping any coulmns

In [ ]:
print(train.dtypes);
print(test.dtypes)

As loan id is not a important feature as it is not affecting the result.So,we will be dropping loan id from both teset and train data.

In [ ]:
train = train.drop("Loan_ID", axis=1)
train.head()

In [ ]:
test_copy = test.copy()

In [ ]:
test_copy = test_copy.drop("Loan_ID", axis=1)
test_copy.head()

As we can see that there are categorial values in Gender,Married,Education and self employed.So, we will be replacing them with number. 

In [ ]:
cleanup_nums = {"Gender":     {"Male": 1, "Female": 2},
                "Married" :  {"Yes": 1, "No": 2},
               "Education" :  {"Graduate": 1, "Not Graduate": 2},
               "Self_Employed" :  {"Yes": 1, "No": 2},
               "Property_Area" : {"Urban" :1, "Semiurban" : 2 ,"Rural" : 3},
               "Loan_Status" : {"Y" : 1, "N" : 0},
               "Dependents": {"0" : 0, "1" : 1, "2" : 2, "3+" : 4}}
train.replace(cleanup_nums, inplace=True)
train.head()



In [ ]:
cleanup_nums = {"Gender":     {"Male": 1, "Female": 2},
                "Married" :  {"Yes": 1, "No": 2},
               "Education" :  {"Graduate": 1, "Not Graduate": 2},
               "Self_Employed" :  {"Yes": 1, "No": 2},
               "Property_Area" : {"Urban" :1, "Semiurban" : 2 ,"Rural" : 3},
               "Dependents": {"0" : 0, "1" : 1, "2" : 2, "3+" : 4}}
test_copy.replace(cleanup_nums, inplace=True)
test_copy.head()


In [ ]:

# split data into X and y
X = train.iloc[:,:-1].values
y = train.iloc[:,11].values


In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.30, random_state=0)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
print(model)

In [ ]:

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


In [ ]:

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


So, using random forest the total accuracy is 78.92%

In [ ]:
test_copy = test_copy.values
test_pred = model.predict(test_copy)


In [ ]:
print(test_pred)

In [ ]:
type(test_pred)

In [ ]:
lg_df = pd.DataFrame({'Loan_Status':test_pred})
lg_df["Loan_Status"]= lg_df["Loan_Status"].replace(1,"Y")
lg_df["Loan_Status"]= lg_df["Loan_Status"].replace(0,"N")
lg_df.head()


In [ ]:
test_new = test.copy()
test_new['Loan_Status'] = lg_df
test_new.head()

In [ ]:
test_new.drop(['Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area'],axis=1,inplace=True)
test_new.head()

In [ ]:
test_new.to_csv('sample_submission_49d68Cx.csv',index=False) 
